In [2]:
from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options 
import time 

from bs4 import BeautifulSoup

In [20]:
url = 'https://www.vseinstrumenti.ru/sadovaya-tehnika/sadoviy-instrument-i-inventar/vedra/otsinkovannye/spets/metall-otsinkovannoe-9l-ved-ots-090/'

In [16]:
url = 'https://www.vseinstrumenti.ru/klimat/ventilyacionnoe-oborudovanie/vytyazhnye-ventilyatory/nakladnye/era-/osevoj-sb-d100-polipropilen-optima-4/'

In [3]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(url)

In [4]:
soup = BeautifulSoup(driver.page_source, "lxml")
print(soup.prettify())


<html class="websockets audio cssgradients contenteditable" data-n-head="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D,%22class%22:%7B%22ssr%22:null%7D%7D" lang="ru">
 <head>
  <script src="https://bam.eu01.nr-data.net/1/NRJS-ec740f33541a4da3aa8?a=328618477&amp;sa=1&amp;v=1212.e95d35c&amp;t=Unnamed%20Transaction&amp;rst=3377&amp;ck=1&amp;ref=https://rostov.vseinstrumenti.ru/product/vedro-spets-metall-otsinkovannoe-9l-ved-ots-09-1473343/&amp;be=1275&amp;fe=3153&amp;dc=1610&amp;perf=%7B%22timing%22:%7B%22of%22:1674726084749,%22n%22:0,%22f%22:248,%22dn%22:248,%22dne%22:248,%22c%22:248,%22ce%22:248,%22rq%22:275,%22rp%22:306,%22rpe%22:1067,%22dl%22:1058,%22di%22:1383,%22ds%22:1610,%22de%22:1610,%22dc%22:3153,%22l%22:3153,%22le%22:3155%7D,%22navigation%22:%7B%7D%7D&amp;fp=1303&amp;fcp=1303&amp;jsonp=NREUM.setToken" type="text/javascript">
  </script>
  <title>
   Ведро СПЕЦ металл, оцинкованное, 9л ВЕД-ОЦ-09 - выгодная цена, отзывы, характеристики, фото - купить в Москве и РФ
  </title>
  <meta cha

In [5]:
driver.close()

In [21]:
options = Options()
options.add_argument('--headless')
# options.add_argument("--window-size=1920,1200")
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36")

driver = webdriver.Chrome(options=options, service=Service(ChromeDriverManager().install()))
driver.get(url)

# time.sleep(5) 
 
driver.save_screenshot("datacamp.png") 
 
# driver.close()
# driver.quit()

True

In [ ]:
driver.page_source

In [29]:
driver.title.find('Ошибка 404')

0

In [31]:
import requests

In [30]:
url = 'https://cdn.vseinstrumenti.ru/images/goods/stroitelnoe-oborudovanie-i-tehnika/oborudovanie-dlya-betonnyh-rabot/1092385/1200x800/54296550.jpg'

In [33]:
resp = requests.get(url)
resp

<Response [200]>

In [35]:
with open('test.jpg', "wb") as f:
    f.write(resp.content)

In [36]:
resp.status_code

200

In [1]:
import os
import csv
import requests
import lxml
import time
import cloudscraper
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from transliterate import slugify
from tqdm import tqdm

In [2]:
cloudscraper.__version__

'1.2.68'

In [2]:
def get_html(url_):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}
    scraper = cloudscraper.create_scraper(delay=10, debug=False,  browser='chrome')
    response = scraper.get(url_, headers=headers)
    return response

def get_prods(url_):
    response = 0
    for i in range(10):
        try:
            response= get_html(url_)
            break
        except:
            time.sleep(1)
            continue
    return response

In [3]:
with open('download.xlsx', "rb") as f:
    df = pd.read_excel(f, dtype=str)
print(df.shape)
df.head()

(1, 4)


,_MODEL_,_SKU_,Name,Url
0,00000000065,00065,Ведро 9л оцин.,https://www.vseinstrumenti.ru/sadovaya-tehnika...


In [4]:
df1 = pd.DataFrame(columns=['_MODEL_', '_SKU_', '_NAME_', '_CATEGORY_', '_MAIN_CATEGORY_', '_MANUFACTURER_', '_DESCRIPTION_', '_ATTRIBUTES_', '_PRODUCT_IMAGES_', '_STATUS_', '_SEO_KEYWORD_'])
df1.to_excel("opencart.xlsx", index = False)

df1 = pd.DataFrame(columns=['_MODEL_', '_SKU_', 'Name', 'Url', 'Where', 'status_code', 'reason'])
df1.to_excel("error.xlsx", index = False)

In [4]:
url_ = df['Url'][0]
url_

'https://www.vseinstrumenti.ru/sadovaya-tehnika/sadoviy-instrument-i-inventar/vedra/otsinkovannye/spets/metall-otsinkovannoe-9l-ved-ots-09/'

In [6]:
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}
scraper = cloudscraper.create_scraper()
# response = scraper.get(url_, headers=headers)
response = scraper.get(url_)

CloudflareChallengeError: Detected a Cloudflare version 2 Captcha challenge, This feature is not available in the opensource (free) version.

In [31]:
for index in tqdm(df.index):
    response = get_prods(df['Url'][index])
    if(response.status_code==200):
        soup = BeautifulSoup(response.text, "lxml")

        _NAME_ = soup.find('h1').text.strip()
        _SEO_KEYWORD_ = slugify(_NAME_)

        breadcrumbs = soup.find('nav', class_="breadcrumbs").text.split('/')
        breadcrumbs = [s.strip() for s in breadcrumbs]
        _CATEGORY_ = '|'.join(map(str, breadcrumbs[1:]))
        _MAIN_CATEGORY_ = _CATEGORY_
        try:
            _MANUFACTURER_ = soup.find('img', class_="brand-logo")['alt']
        except:
            _MANUFACTURER_ = ""
        try:
            _DESCRIPTION_ = soup.find(itemprop="description").text.strip()
        except:
            _DESCRIPTION_ = ""

        # Find attributes
        tag_names = soup.find(class_='specification-list').findAll(itemprop='name')
        tag_values = soup.find(class_='specification-list').findAll(itemprop='value')
        _ATTRIBUTES_ = ''
        for i in range(len(tag_names)):
            _ATTRIBUTES_ = _ATTRIBUTES_ + 'Характеристики' + '|' + tag_names[i].text.strip() + '|' + tag_values[i].text.strip() + '\n'
        
        # Find images link
        images = soup.find(class_='carousel -vertical').find_all('a')
        big_img = []
        for img in images:
            big_img.append(img['href'])
        
        # Download images
        local_path = []
        for url_img in big_img:
            resp = get_prods(url_img)
            path = 'catalog/' + '/'.join(map(str, url_img.split('/')[4:]))
            local_path.append(path)
            # Save the image
            if (type(resp)!=int and resp.status_code == 200):
                os.makedirs(os.path.dirname(path), exist_ok=True)
                with open(path, "wb") as f:
                    f.write(resp.content)
            else:
                df1 = pd.DataFrame([[df['_MODEL_'][index], df['_SKU_'][index], df['Name'][index],  df['Url'][index], 'Images', resp.status_code, resp.reason]])
                with pd.ExcelWriter('error.xlsx', mode='a', if_sheet_exists="overlay") as writer:  
                    df1.to_excel(writer, startrow =writer.sheets['Sheet1'].max_row, index = False, header = False)
            
            _PRODUCT_IMAGES_ = ','.join(map(str, local_path))
        
        _STATUS_ = '1'
        
        df1 = pd.DataFrame([[df['_MODEL_'][index], df['_SKU_'][index], _NAME_, _CATEGORY_, _MAIN_CATEGORY_, _MANUFACTURER_, _DESCRIPTION_, _ATTRIBUTES_, _PRODUCT_IMAGES_, _STATUS_, _SEO_KEYWORD_]])
        with pd.ExcelWriter('opencart.xlsx', mode='a', if_sheet_exists="overlay") as writer:  
            df1.to_excel(writer, startrow =writer.sheets['Sheet1'].max_row, index = False, header = False)
    else:
        df1 = pd.DataFrame([[df['_MODEL_'][index], df['_SKU_'][index], df['Name'][index],  df['Url'][index], 'Url', response.status_code, response.reason]])
        with pd.ExcelWriter('error.xlsx', mode='a', if_sheet_exists="overlay") as writer:  
            df1.to_excel(writer, startrow =writer.sheets['Sheet1'].max_row, index = False, header = False)

  0%|          | 0/1 [00:11<?, ?it/s]


AttributeError: 'int' object has no attribute 'status_code'

In [32]:
big_img

['https://cdn.vseinstrumenti.ru/images/goods/sadovaya-tehnika-i-instrument/sadovyj-inventar-i-instrument/1473343/1200x800/51302289.jpg',
 'https://cdn.vseinstrumenti.ru/images/goods/sadovaya-tehnika-i-instrument/sadovyj-inventar-i-instrument/1473343/1200x800/54359172.jpg',
 'https://cdn.vseinstrumenti.ru/images/goods/sadovaya-tehnika-i-instrument/sadovyj-inventar-i-instrument/1473343/1200x800/54359173.jpg',
 'https://cdn.vseinstrumenti.ru/images/goods/sadovaya-tehnika-i-instrument/sadovyj-inventar-i-instrument/1473343/1200x800/54359174.jpg']

In [34]:
response

0

In [ ]:
with open('opencart.csv', 'w', newline='', encoding="utf-8") as f:
  write = csv.writer(f, delimiter=';', quoting=csv.QUOTE_ALL)   
  write.writerow(['_MODEL_', '_SKU_', '_NAME_', '_CATEGORY_', '_MAIN_CATEGORY_', '_MANUFACTURER_', '_DESCRIPTION_', '_ATTRIBUTES_', '_PRODUCT_IMAGES_', '_STATUS_', '_SEO_KEYWORD_'])

with open('error.csv', 'w', newline='', encoding="utf-8") as f:
  write = csv.writer(f, delimiter=';', quoting=csv.QUOTE_ALL)   
  write.writerow(['_MODEL_', '_SKU_', 'Name', 'Url', 'Where', 'status_code', 'reason'])

In [ ]:
with open('download.xlsx', "rb") as f:
    df = pd.read_excel(f, dtype=str)
df

,_MODEL_,_SKU_,Name,Url
0,00000015295,УШМ-115-800 М3,"Углошлифовальная машина ЗУБР УШМ-115-800 М3, 1...",https://www.vseinstrumenti.ru/instrument/shlif...
1,00000015296,УШМ-125-1100 ТМ3,Углошлифовальная машина ЗУБР УШМ-125-1100 ТМ3 ...,https://www.vseinstrumenti.ru/instrument/shlif...
2,00000015302,УШМ-125-1200 ЭМЗ,"Углошлифовальная машина ЗУБР УШМ-125-1200, 125...",https://www.vseinstrumenti.ru/instrument/shlif...


In [ ]:
for index in tqdm(df.index):
    response = get_prods(df['Url'][index])
    if(response.status_code==200):
        soup = BeautifulSoup(response.text, "lxml")

        _NAME_ = soup.find('h1').text.strip()
        _SEO_KEYWORD_ = slugify(_NAME_)

        breadcrumbs = soup.find('nav', class_="breadcrumbs").text.split('/')
        breadcrumbs = [s.strip() for s in breadcrumbs]
        _CATEGORY_ = '|'.join(map(str, breadcrumbs[1:]))
        _MAIN_CATEGORY_ = _CATEGORY_
        _MANUFACTURER_ = soup.find('img', class_="brand-logo")['alt']
        _DESCRIPTION_ = soup.find(itemprop="description").text.strip()

        # Find attributes
        tag_names = soup.find(class_='features spoiler').findAll(itemprop='name')
        tag_values = soup.find(class_='features spoiler').findAll(itemprop='value')
        _ATTRIBUTES_ = ''
        for i in range(len(tag_names)):
            _ATTRIBUTES_ = _ATTRIBUTES_ + 'Характеристики' + '|' + tag_names[i].text.strip() + '|' + tag_values[i].text.strip() + '\n'
        
        # Find images link
        images = soup.find(class_='listing-carousel').find_all('a')
        big_img = []
        for img in images:
            zoom = img.find('div', class_='zoom')
            responses = zoom['style'].split("url('")[1][:-2]
            if(zoom.find_next_sibling('div')==None):
                big_img.append(responses)
        
        # Download images
        local_path = []
        for url_img in big_img:
            resp = get_prods(url_img)
            path = 'catalog/' + '/'.join(map(str, url_img.split('/')[4:]))
            local_path.append(path)
            # Save the image
            if resp.status_code == 200:
                os.makedirs(os.path.dirname(path), exist_ok=True)
                with open(path, "wb") as f:
                    f.write(resp.content)
            else:
                with open('error.csv', 'a', newline='') as f:
                    write = csv.writer(f, delimiter=';')   
                    write.writerow([df['_MODEL_'][index], df['_SKU_'][index], df['Name'][index],  df['Url'][index], 'Images', resp.status_code, resp.reason])
            
            _PRODUCT_IMAGES_ = ','.join(map(str, local_path))
        
        _STATUS_ = '1'

        with open('opencart.csv', 'a', newline='', encoding="utf-8") as f:
            write = csv.writer(f, delimiter=';', quoting=csv.QUOTE_NONNUMERIC)   
            write.writerow([df['_MODEL_'][index], df['_SKU_'][index], _NAME_, _CATEGORY_, _MAIN_CATEGORY_, _MANUFACTURER_, _DESCRIPTION_, _ATTRIBUTES_, _PRODUCT_IMAGES_, _STATUS_, _SEO_KEYWORD_])
    else:
        with open('error.csv', 'a', newline='', encoding="utf-8") as f:
            write = csv.writer(f, delimiter=';', quoting=csv.QUOTE_NONNUMERIC)   
            write.writerow([df['_MODEL_'][index], df['_SKU_'][index], df['Name'][index],  df['Url'][index], 'Url', response.status_code, response.reason])

100%|██████████| 3/3 [00:11<00:00,  3.90s/it]
